In [1]:
import twitter
api_key = "hSRo8Qzpisjny0tHxiwgLmARc"
secret_key = "sjchcV9k6cVKL8tvhmoWAZk0X1goayiIyZF272zSTXoT10ZjNO"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAMdWPAEAAAAAtKCKcmghy%2BtURo1%2FNSu642EC6pk%3DnIdNiQZLHBSMzgpUbBldmtNbi8tDLQCoDO3Zbm9ULTtHMpCAwc"
access_token = "1327408007412637696-Op2eyy7CdeoMwWL4t3hyka93j2Nsn2"
access_token_secret = "laX2RvuanrbWOsRsJ78jzYoIXGZl8f68tD52uzZKwm3v0"

api = twitter.Api(consumer_key=api_key, consumer_secret=secret_key, access_token_key=access_token, access_token_secret=access_token_secret)

In [2]:
def get_tweets(api=None, screen_name=None):
    timeline = api.GetUserTimeline(screen_name=screen_name, count=200)
    earliest_tweet = min(timeline, key=lambda x: x.id).id
    print("getting tweets before:", earliest_tweet)

    while True:
        try:
            tweets = api.GetUserTimeline(
                screen_name=screen_name, max_id=earliest_tweet, count=200
            )
            new_earliest = min(tweets, key=lambda x: x.id).id

            if not tweets or new_earliest == earliest_tweet:
                break
            else:
                earliest_tweet = new_earliest
                print("getting tweets before:", earliest_tweet)
                timeline += tweets
        except:
            break

    return timeline


In [3]:
names = ['elonmusk', 'JoeBiden', 'sundarpichai', 'tim_cook']
timelines = {}
for name in names:
    timeline = get_tweets(api=api, screen_name=name)
    timelines[name] = timeline

ValueError: min() arg is an empty sequence

In [47]:
timelines['elonmusk'][0]

Status(ID=1387901003664699392, ScreenName=elonmusk, Created=Thu Apr 29 22:45:59 +0000 2021, Text='A major part of real-world AI has to be solved to make unsupervised, generalized full self-driving work, as the ent… https://t.co/eRFEqUqjV0')

In [51]:
symbols = ['bitcoin', 'doge', 'tesla', 'google', 'apple']
df_columns = ['user', 'timestamp', 'tweet text', 'tweet sentiment', 'link', 'included symbols'] + [x + " Returns" for x in symbols]

In [43]:
symbol_synonyms = {}
symbol_synonyms['bitcoin'] = ['btc']
symbol_synonyms['doge'] = ['dogecoin']
symbol_synonyms['tesla'] = ['tsla']
symbol_synonyms['google'] = ['goog', 'alphabet', "youtube"]
symbol_synonyms['apple'] = ['aapl', 'mac', 'm1']

In [30]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Nick\AppData\Roaming\nltk_data...


In [52]:
import pandas as pd
df = pd.DataFrame(columns = df_columns)

In [53]:
for user in timelines:
    for tweet in timelines[user]:
        sent = sia.polarity_scores(tweet.text)
        symbols_included = []
        for symbol in symbols:
            if symbol in tweet.text.lower():
                symbols_included.append(symbol)
                continue
            for syn in symbol_synonyms[symbol]:
                if syn in tweet.text.lower():
                    symbols_included.append(symbol)
                    break
        dat = [user, tweet.created_at.split(' +')[0], tweet.text, sent['compound'], "twitter.com/{:s}/status/{:s}".format(user, str(tweet.id)),symbols_included] + [0 for sym in symbols]

        df.loc[len(df)] = dat

In [4]:
df

NameError: name 'df' is not defined

In [56]:
df.to_csv("./sample.csv")